<a href="https://colab.research.google.com/github/reggy0/leaf_detection/blob/main/Cotton_Disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!nvidia-smi

Thu Dec 10 11:55:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    31W /  70W |    997MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.3.0'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

train_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/kaggle_dataset/cotton-disease-dataset/Cotton Disease/train',
                                              target_size=(64,64),
                                              batch_size =32,
                                              class_mode='categorical')

Found 1951 images belonging to 4 classes.


In [ ]:
train_set.class_indices

{'diseased cotton leaf': 0,
 'diseased cotton plant': 1,
 'fresh cotton leaf': 2,
 'fresh cotton plant': 3}

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory('/content/gdrive/MyDrive/kaggle_dataset/cotton-disease-dataset/Cotton Disease/test',
                                              target_size=(64,64),
                                              batch_size =32,
                                              class_mode='categorical')

Found 106 images belonging to 4 classes.


In [ ]:
test_set.class_indices

{'diseased cotton leaf': 0,
 'diseased cotton plant': 1,
 'fresh cotton leaf': 2,
 'fresh cotton plant': 3}

In [ ]:
from tensorflow.keras.layers import Conv2D

In [ ]:
cnn = tf.keras.models.Sequential()

cnn.add(tf.keras.layers.Conv2D(filters=32, padding="same", kernel_size=3, activation='relu', input_shape=[64,64,3]))

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Flatten())

cnn.add(tf.keras.layers.Dense(units=128, activation="relu"))

cnn.add(tf.keras.layers.Dense(units=4, activation="softmax"))

In [ ]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1048704   
_________________________________________________________________
dense_3 (Dense)              (None, 4)                

Training

In [ ]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn.fit(x=train_set, validation_data=test_set,epochs=10)

Epoch 1/10
61/61 [==============================] - 497s 8s/step - loss: 1.2237 - accuracy: 0.4813 - val_loss: 1.0369 - val_accuracy: 0.5849
Epoch 2/10
61/61 [==============================] - 18s 303ms/step - loss: 0.8750 - accuracy: 0.6699 - val_loss: 0.9053 - val_accuracy: 0.6132
Epoch 3/10
61/61 [==============================] - 19s 304ms/step - loss: 0.6651 - accuracy: 0.7514 - val_loss: 0.7858 - val_accuracy: 0.6604
Epoch 4/10
61/61 [==============================] - 19s 308ms/step - loss: 0.5957 - accuracy: 0.7611 - val_loss: 0.5005 - val_accuracy: 0.7925
Epoch 5/10
61/61 [==============================] - 19s 305ms/step - loss: 0.5558 - accuracy: 0.7914 - val_loss: 0.5529 - val_accuracy: 0.7830
Epoch 6/10
61/61 [==============================] - 19s 308ms/step - loss: 0.4886 - accuracy: 0.8114 - val_loss: 0.5753 - val_accuracy: 0.7358
Epoch 7/10
61/61 [==============================] - 18s 300ms/step - loss: 0.4607 - accuracy: 0.8283 - val_loss: 0.3862 - val_accuracy: 0.8868
E

In [ ]:
from tensorflow.keras.models import load_model

cnn.save('/content/gdrive/MyDrive/Colab Notebooks/Cotton Disease Classification/model_rcotton_disease.h5')

## Single Prediction

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_img = image.load_img('/content/gdrive/MyDrive/kaggle_dataset/cotton-disease-dataset/Cotton Disease/test/fresh cotton leaf/d (133)_iaip.jpg',target_size=(64,64))
test_img = image.img_to_array(test_img)
test_img = test_img/255
test_img = np.expand_dims(test_img, axis = 0)
result = cnn.predict(test_img)

In [ ]:
predict = np.argmax(result)

In [ ]:
if predict == 0:
  print("This leaf is diseased cotton leaf")
elif predict == 1:
  print("This leaf is diseased cotton plant")
elif predict == 2:
  print("This leaf is fresh cotton leaf")
else:
  print("This leaf is fresh cotton plant")

This leaf is fresh cotton leaf
